## Extract Documents from database and generate questions based on chunk number

In [ ]:
import logging
import ipywidgets as widgets

from notebooks.question_answer_generator.generate_q_a_pairs import process_document_or_chunk, write_question_answer_json_to_csv
from src.db.models.document import Document
from src.app_config import app_config

logger = logging.getLogger(__name__)

num_qa_per_chunk_or_doc = widgets.IntText(
    value=5,
    description='Number of questions per chunk/document:',
    disabled=False   
)
display(num_qa_per_chunk_or_doc)

llm_model = widgets.Dropdown(
    options=['gpt-3.5-turbo-instruct', 'gpt-4o', 'gpt-4o-mini'],
    value='gpt-4o',
    description='OpenAI LLM Model:',
    disabled=False,
)
display(llm_model)

IntText(value=5, description='Number of questions per chunk/document:')

Dropdown(description='OpenAI LLM Model:', index=1, options=('gpt-3.5-turbo-instruct', 'gpt-4o', 'gpt-4o-mini')…

In [19]:
question_gen_selection = widgets.RadioButtons(
    options=['document', 'chunk'],
    value='document',
    description='Question source:',
    disabled=False
)
display(question_gen_selection)

RadioButtons(description='Question source:', options=('document', 'chunk'), value='document')

In [20]:
question_dataset = widgets.SelectMultiple(
    options=['Imagine LA', 'CA EDD', 'BEM'],
    value=['Imagine LA', 'CA EDD' ],
    description='Dataset source:',
    disabled=False
)
display(question_dataset)

SelectMultiple(description='Dataset source:', index=(0, 1), options=('Imagine LA', 'CA EDD', 'BEM'), value=('I…

In [ ]:
with app_config.db_session() as db_session:
    selected_dataset = list(question_dataset.value)
    documents = db_session.query(Document).filter(Document.dataset.in_(selected_dataset)).all()

    fields = ["question", "answer", "document_name", "document_source", "dataset", "document_id", "chunk_id"]

    logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
    logging.info(f'Start processing with llm {llm_model.value}')

    dataset_list = ", ".join(question_dataset.value)
    for document in documents:
        chunk_list= document.chunks
        if question_gen_selection.value == "chunk":
            for chunk in chunk_list:
                chunk_q_a_json = process_document_or_chunk(document=chunk, num_of_chunks=num_qa_per_chunk_or_doc.value, llm=llm_model.value, dataset=dataset_list)
                write_question_answer_json_to_csv("question_answer_pairs.csv", fields, chunk_q_a_json)        
        else:
            document_q_a_json = process_document_or_chunk(document=document, num_of_chunks=num_qa_per_chunk_or_doc.value,llm=llm_model.value, dataset=dataset_list)
            write_question_answer_json_to_csv("question_answer_pairs.csv", fields, document_q_a_json)        
    logger.info("Finished processing")

2024-11-26 19:03:19,193 - INFO - Constructed database configuration
2024-11-26 19:03:19,212 - INFO - connected to postgres db
2024-11-26 19:03:19,212 - WARNING - database connection is not using SSL


ProgrammingError: (psycopg.errors.UndefinedFunction) operator does not exist: text && character varying
LINE 3: WHERE document.dataset && $1::VARCHAR
                               ^
HINT:  No operator matches the given name and argument types. You might need to add explicit type casts.
[SQL: SELECT document.name AS document_name, document.content AS document_content, document.source AS document_source, document.dataset AS document_dataset, document.program AS document_program, document.region AS document_region, document.id AS document_id, document.created_at AS document_created_at, document.updated_at AS document_updated_at 
FROM document 
WHERE document.dataset && %(dataset_1)s::VARCHAR]
[SQL parameters hidden due to hide_parameters=True]
(Background on this error at: https://sqlalche.me/e/20/f405)